# Q-Learning

Date: 2019-09-26  
Author: skettee  
Categories: Reinforcement Learning, Q-Learning    
Tags: Environment, Agent, State, Action, Reward, Policy, Q-value, 𝜀-greedy    
<!--eofm-->

강화 학습(Reinforcement Learning)에서 사용하는 Q-Learning에 대해서 알아 보고 Gym에서 제공하는 문제를 해결하기 위한 알고리듬을 만들어 보자.  
<!--more-->

실제로 돌려 보고 싶으면 구글 코랩으로 ~  

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/skettee/notebooks/blob/master/q_learning.ipynb)

👤 상사

> SARSA에서 마지막 A를 수행을 안해도 되는 방법이 있다고 하네?        
> 아래 체육관(Gym)에 가서      
> '얼음 호수8X8' 문제를 그걸로 풀어 보게          
>
> https://gym.openai.com/envs/FrozenLake8x8-v0/

⚙️ 엔지니어

> 네~ 네~


## 문제 분석 (Problem Anaysis)

'얼음 호수8X8'은 살사(SARSA)에서 돌려 보았던 문제다. 이번에는 미끄러 지는 것을 추가해 보자!   

In [1]:
import gym
import numpy as np
from time import sleep
from IPython.display import display, clear_output, Pretty

In [2]:
#
# Environment
#
env = gym.make('FrozenLake8x8-v0')
state = env.reset()

# Initial world display
world = env.render(mode='ansi')
display(Pretty(world))
sleep(0.5)

#
# Agent
#
for step in range(100):
    action =env.action_space.sample()
    next_state, reward, done, info = env.step(action)    
    state = next_state
    
    # updated world display
    world = env.render(mode='ansi')
    clear_output(wait=True)
    display(Pretty(world))
    sleep(0.5)
    
    if done: # an episode finished
        print("Episode finished after {} timesteps".format(step+1))
        break

  (Down)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG


Episode finished after 17 timesteps


⚙️ 엔지니어

> 가끔 액션과는 다른 방향으로 가버린다.   
> 과연  미끄러져서 구멍에 빠지지 않고    
> 목표점에 도달할 수 있을까?   

## 환경 (Environment)

'얼음호수8X8' 세계의 환경은 64개의 상태(State)와 4개의 액션(Action)으로 구성 되어 있다.  그리고 아래를 보자    

In [3]:
env.P[55]

{0: [(0.3333333333333333, 47, 0.0, False),
  (0.3333333333333333, 54, 0.0, True),
  (0.3333333333333333, 63, 1.0, True)],
 1: [(0.3333333333333333, 54, 0.0, True),
  (0.3333333333333333, 63, 1.0, True),
  (0.3333333333333333, 55, 0.0, False)],
 2: [(0.3333333333333333, 63, 1.0, True),
  (0.3333333333333333, 55, 0.0, False),
  (0.3333333333333333, 47, 0.0, False)],
 3: [(0.3333333333333333, 55, 0.0, False),
  (0.3333333333333333, 47, 0.0, False),
  (0.3333333333333333, 54, 0.0, True)]}

해석을 해보면 다음과 같다.

> 55번 상태(state)에서,  
> 왼쪽으로 이동하라는 액션을 주면, 1/3의 확률로 위로 이동하고,  1/3의 확률로 왼쪽으로 이동하고, 1/3의 확률로 아래로 이동한다.  
> 아래로 이동하라는 액션을 주면, 1/3의 확률로 왼쪽으로 이동하고, 1/3의 확률로 아래로 이동하고, 1/3의 확률로 그 자리에 있는다.  
> 오른쪽으로 이동하라는 액션을 주면, 1/3의 확률로 아래로 이동하고, 1/3의 확률로 그자리에 있고, 1/3의 확률로 위로 이동한다.  
> 아래로 이동하라는 액션을 주면, 1/3의 확률로 그 자리에 있고, 1/3의 확률로 위로 이동하고, 1/3의 확률로 왼쪽으로 이동한다.  

이렇게 미끄러짐이 추가 되면 1/3의 확률로 정상적인 액션을 수행한다.  


## Q-Learning

Q-Learning 의 원리는 살사(SARSA)에서 서로 다른 정책(policy)으로 훈련 시킨 데이터를 섞어도 최적화가 가능하다는 것이다.  

일단, SARSA + greedy 방식으로 훈련해서 $Q(S, A')$을 만들어 놓는다.

그리고 우리는 SARSA + 𝜀-greedy 방식으로 훈련해서 $Q(S, A)$를 만들 것이다.  이렇게...

$Q(S_t, A_t) \leftarrow Q(S_t, A_t) + \alpha \left( R_{t+1} + \gamma Q(S_{t+1}, A_{t+1}) - Q(S_t, A_t) \right)$

여기서 $Q(S_{t+1}, A_{t+1})$ 대신에 SARSA + greedy 방식으로 만들어진 $Q(S_{t+1}, A')$을 사용한다.  이렇게...  

$Q(S_t, A_t) \leftarrow Q(S_t, A_t) + \alpha \left( R_{t+1} + \gamma Q(S_{t+1}, A') - Q(S_t, A_t) \right)$

A' 은 greedy 방식으로 만들어 졌으므로 다음의 식이 성립한다.  

$\pi(S_{t+1}) = \text{argmax}_{a'} Q(S_{t+1}, a')$

$\pi(S_{t+1})$를 A' 에 박아 넣으면 다음의 식이 나온다.  


$\mathsf {Q(S, A) \leftarrow Q(S, A) + \alpha \left( R + \gamma \max_{a'} Q(S', a') - Q(S, A) \right)}$

$\alpha$ : learning rate  
$\gamma$ : 디스카운트 (discount factor)  


⚙️ 엔지니어

> SARSA에서 마지막 A를 수행할 필요 없이     
> Q 테이블에서 가장 높을 점수의 액션을 선택하면   
> 최적화가 가능하다는 것이 바로   
> Q-Learning이다!  

## 학습 (Learning)

Q-Learning을 이용해서 최적의 Q-value를 찾아보자!   

SARSA보다 코드가 깔끔하다!

In [4]:
from tqdm import tqdm

num_state = env.observation_space.n
num_action = env.action_space.n
num_episode = 5000

# Initialize Q_table 
Q_table = np.random.uniform(low=0.0, high=0.00000001, size=(num_state, num_action))
# Zero for terminate states
for s in [19, 29, 35, 41, 42, 49, 52, 54, 59, 63]:
    Q_table[s] = 0

for episode in tqdm(range(num_episode)):
    state = env.reset()
    done = False
    # Hyper parameter
    epsilon = 0.3
    alpha = 0.1
    gamma = 0.9
    
    while not done:
        if np.random.uniform() < epsilon:
            action = env.action_space.sample()
        else:
            action = np.argmax(Q_table[state])
        next_state, reward, done, info = env.step(action)
        Q_table[state][action] += alpha * (reward + gamma*Q_table[next_state, np.argmax(Q_table[next_state])] \
                                           - Q_table[state][action])
        state = next_state


100%|██████████| 5000/5000 [00:02<00:00, 1558.53it/s]


## 해결 (Solution)

In [6]:
state = env.reset()
done = False

# Initial world display
world = env.render(mode='ansi')
display(Pretty(world))
sleep(0.5)

while not done:
    action = np.argmax(Q_table[state]) # Optimal Policy
    state, reward, done, info = env.step(action)
    
    # updated world display
    world = env.render(mode='ansi')
    clear_output(wait=True)
    display(Pretty(world))
    sleep(0.5)
    
    if done and state == 63:
        print('\n 🎉👍 성공! 🍺🥇')
            

  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG



 🎉👍 성공! 🍺🥇
